<a href="https://colab.research.google.com/github/wenchun0731/github-colab/blob/main/FineTune(YOLOV8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#掛載雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#下載ultralytics開源庫
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 14.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
#下載YOLOv8並檢查模型配置
from ultralytics import YOLO
import os
from IPython.display import display,Image
from IPython import display
display.clear_output()#清除当前输出单元格的所有输出内容
!yolo model=checks#检查 YOLO 模型的状态或配置信息。

WARNING ⚠️ 'mode' argument is missing. Valid modes are {'val', 'train', 'track', 'benchmark', 'predict', 'export'}. Using default 'mode=train'.
Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 548, in entrypoint
    stem = Path(model).stem.lower()
  File "/usr/lib/python3.10/pathlib.py", line 960, in __new__
    self = cls._from_parts(args)
  File "/usr/lib/python3.10/pathlib.py", line 594, in _from_parts
    drv, root, parts = self._parse_args(args)
  File "/usr/lib/python3.10/pathlib.py", line 578, in _parse_args
    a = os.fspath(a)
TypeError: expected str, bytes or os.PathLike object, not module


In [ ]:
#下載roboflow(這個要跑)
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0


In [ ]:
# 匯入資料集
from roboflow import Roboflow
rf = Roboflow(api_key="b2dEbFft2r35WYNkzDD5")
project = rf.workspace("test-lmhkl").project("car-traffic-light-licence")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.196`, to intall it `pip install ultralytics==8.0.196`.



Extracting Dataset Version Zip to car、traffic-light、licence-1 in yolov8:: 100%|██████████| 1512/1512 [00:00<00:00, 5029.78it/s]


In [ ]:
#訓練模式(相當於train)
!yolo task=detect mode=train model=yolov8m.pt data={dataset.location}/data.yaml batch 24 epochs=20 imgsz=640 project=/content/drive/MyDrive/Finetune/Licence_Finetune/train

In [ ]:
#斷點訓練
!yolo train --resume last.pt data=your_dataset.yaml --batch 48 --epochs=20 imgsz=640 --project /content/drive/MyDrive/Finetune/Licence_Finetune/train
#!yolo train data=your_dataset.yaml model=last.pt epochs=total_epochs_you_want

In [ ]:
#驗證模式(在指定的驗證數據集上進行評估，以檢測模型的性能和準確度)
!yolo task=detect mode=val model=best.pt檔位置 data={dataset.location}/data.yaml

In [ ]:
#預測模式(將使用已訓練的權重文件來對新圖像進行對象檢測，相當於detect)
!yolo task=detect mode=predict model=best.pt檔位置 source=圖片位置 imgsz=640

In [ ]:
#Deep sort
#DeepSort(使用YOLOV8預訓練檔)
!pip install ultralytics
!pip install deep-sort-realtime
import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab.patches import cv2_imshow

model = YOLO('yolov8l.pt')
#
cap = cv2.VideoCapture("/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/test_video.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fNUMS = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/Finetune/Licence_Finetune/detect-log/test-video.mp4", fourcc, fps, size)

tracker = DeepSort(max_age=5)

def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=1, lineType=cv2.LINE_AA)
    if label:
        w, h = cv2.getTextSize(label, 0, fontScale=2 / 3, thickness=1)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image,
                    label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2/3, txt_color, thickness=1, lineType=cv2.LINE_AA)

while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model(frame, conf=0.4)
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                if output[5] == 2:
                    detections.append(([x1, y1, int(x2-x1), int(y2-y1)], output[4], 'car'))
                elif output[5] == 5:
                    detections.append(([x1, y1, int(x2-x1), int(y2-y1)], output[4], 'bus'))
                elif output[5] == 7:
                    detections.append(([x1, y1, int(x2-x1), int(y2-y1)], output[4], 'truck'))

            tracks = tracker.update_tracks(detections, frame=frame)

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                box_label(frame, bbox, '#' + str(int(track_id)) + track.det_class, (167, 146, 11))

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()